# Milestone 2

In [ ]:
"""
air_quality_pipeline.py

Pipeline:
1. Imports
2. Data cleaning + EDA (saves cleaned CSV)
3. Train/test split (time-based)
4. Train ARIMA/Prophet/XGBoost/LSTM (if packages available)
5. Evaluate MAE & RMSE and pick best model per group
6. Save best model(s) for inference (joblib / model.save)
7. Handles per-city / per-station models if a grouping column is found
"""

import os, math, traceback, warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from pathlib import Path



# -------- Config --------
DATA_PATH = "air_quality_data.csv"   # change if needed
OUT_DIR = "./artifacts"
CLEANED_PATH = os.path.join(OUT_DIR, "air_quality_data_cleaned.csv")
BEST_MODELS_PATH = os.path.join(OUT_DIR, "best_models_per_group.csv")
os.makedirs(OUT_DIR, exist_ok=True)

# -------- Utilities --------
def mae(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return float(np.mean(np.abs(y_true - y_pred)))

def rmse(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return float(np.sqrt(np.mean((y_true - y_pred)**2)))

def time_train_test_split(series, test_size=0.2):
    n = len(series)
    split = int(n * (1 - test_size))
    train = series.iloc[:split]
    test = series.iloc[split:]
    return train, test

# -------- Load data --------
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"CSV file not found at {DATA_PATH}. Update DATA_PATH accordingly.")
df = pd.read_csv(DATA_PATH)
print("Loaded CSV:", DATA_PATH, " shape:", df.shape)
print("Columns:", df.columns.tolist())

# -------- Detect datetime column --------
datetime_col = None
for c in df.columns:
    if any(k in c.lower() for k in ("date","time","timestamp")):
        datetime_col = c
        break
if datetime_col is None:
    # try to find object column that parses to datetime
    for c in df.columns:
        if df[c].dtype == object:
            sample = df[c].dropna().astype(str).head(20).tolist()
            ok = True
            for s in sample:
                try:
                    pd.to_datetime(s)
                except:
                    ok = False
                    break
            if ok:
                datetime_col = c
                break
print("Detected datetime column:", datetime_col)

if datetime_col:
    df[datetime_col] = pd.to_datetime(df[datetime_col], errors="coerce")
    df = df.dropna(subset=[datetime_col]).sort_values(by=datetime_col).reset_index(drop=True)
    df = df.set_index(datetime_col)
else:
    print("No datetime column detected — will use integer index. Time-series models expect a time index for best results.")

# -------- Detect group column (city/station) --------
group_col = None
for c in df.columns:
    if any(k in c.lower() for k in ("city","station","location","site")):
        group_col = c
        break
print("Detected group column:", group_col)

# -------- Detect target pollutant column --------
poll_candidates = [c for c in df.columns if any(p in c.lower() for p in ("pm2.5","pm25","pm10","pm 2.5","pm_10","pm_25","pm"))]
if not poll_candidates:
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    poll_candidates = [c for c in numeric_cols if c != group_col]
if not poll_candidates:
    raise ValueError("No numeric pollutant-like columns found.")
target = max(poll_candidates, key=lambda c: df[c].notna().sum())
print("Selected target column:", target)

# -------- EDA summary (minimal) --------
print("\n--- EDA summary for target ---")
print(df[[target]].describe(percentiles=[0.01,0.05,0.25,0.5,0.75,0.95,0.99]))
print("\nNull counts per column:")
print(df.isna().sum())

# -------- Cleaning --------
cleaned = df.copy()
if isinstance(cleaned.index, pd.DatetimeIndex):
    # if high freq (sub-daily) -> resample to daily mean; else set daily freq
    try:
        delta = cleaned.index.to_series().diff().median()
        if pd.notnull(delta) and delta < pd.Timedelta(days=1):
            cleaned = cleaned.resample('D').mean()
            print("Resampled to daily mean.")
        else:
            cleaned = cleaned.asfreq('D')
            print("Set index frequency to daily (may introduce NaNs).")
    except Exception as e:
        print("Resample skipped:", e)
num_cols = cleaned.select_dtypes(include=[np.number]).columns.tolist()
if isinstance(cleaned.index, pd.DatetimeIndex):
    cleaned[num_cols] = cleaned[num_cols].interpolate(method='time', limit_direction='both')
else:
    cleaned[num_cols] = cleaned[num_cols].interpolate().ffill().bfill()
cleaned = cleaned.dropna(subset=[target])
cleaned.to_csv(CLEANED_PATH)
print("Saved cleaned dataset to:", CLEANED_PATH)

# -------- Setup groups --------
if group_col and group_col in cleaned.columns:
    groups = cleaned[group_col].dropna().unique().tolist()
    if len(groups) == 0:
        groups = [None]
else:
    groups = [None]
print("Groups detected:", groups[:10])

# -------- Optional libs detection (import only if available) --------
_has_joblib = True
try:
    import joblib
except:
    _has_joblib = False

def safe_import(name):
    try:
        module = __import__(name)
        return module
    except Exception as e:
        return None

pmd = safe_import("pmdarima")
statsmodels = safe_import("statsmodels")
prophet_mod = safe_import("prophet") or safe_import("fbprophet")
xgb_mod = safe_import("xgboost")
tf_mod = safe_import("tensorflow")

print("Libraries available: pmdarima:", bool(pmd), "statsmodels:", bool(statsmodels),
      "prophet:", bool(prophet_mod), "xgboost:", bool(xgb_mod), "tensorflow:", bool(tf_mod), "joblib:", _has_joblib)

# -------- Modeling loop per group --------
results = []
for grp in groups:
    try:
        if grp is None:
            s_df = cleaned[[target]].dropna().copy()
            gname = "global"
        else:
            s_df = cleaned[cleaned[group_col] == grp][[target]].dropna().copy()
            gname = str(grp)
        if len(s_df) < 10:
            print(f"Skipping group {gname}: insufficient data ({len(s_df)} rows).")
            continue
        if not isinstance(s_df.index, pd.DatetimeIndex):
            s_df.index = pd.date_range(start="2000-01-01", periods=len(s_df), freq='D')
        train, test = time_train_test_split(s_df[target], test_size=0.2)
        print(f"\nGroup {gname}: train={len(train)} test={len(test)}")
        model_scores = {}

        # --- ARIMA (pmdarima preferred) ---
        try:
            if pmd is not None:
                print("Fitting pmdarima auto_arima...")
                ar = pmd.arima.auto_arima(train, seasonal=False, error_action='ignore', suppress_warnings=True)
                preds = ar.predict(n_periods=len(test))
                m_mae, m_rmse = mae(test.values, preds), rmse(test.values, preds)
                ar_path = os.path.join(OUT_DIR, f"arima_{gname}.pkl")
                if _has_joblib:
                    joblib.dump(ar, ar_path)
                model_scores["ARIMA"] = (m_mae, m_rmse, ar_path)
                print("ARIMA done:", m_mae, m_rmse)
            elif statsmodels is not None:
                import statsmodels.api as sm
                from statsmodels.tsa.statespace.sarimax import SARIMAX
                print("Fitting SARIMAX(1,1,1) fallback...")
                m = SARIMAX(train, order=(1,1,1), enforce_stationarity=False, enforce_invertibility=False)
                res = m.fit(disp=False)
                preds = res.get_forecast(steps=len(test)).predicted_mean
                m_mae, m_rmse = mae(test.values, preds.values), rmse(test.values, preds.values)
                ar_path = os.path.join(OUT_DIR, f"sarimax_{gname}.pkl")
                if _has_joblib:
                    joblib.dump(res, ar_path)
                model_scores["ARIMA"] = (m_mae, m_rmse, ar_path)
                print("SARIMAX done:", m_mae, m_rmse)
            else:
                print("ARIMA skipped (no pmdarima or statsmodels).")
        except Exception as e:
            print("ARIMA error:", e)

        # --- Prophet ---
        try:
            if prophet_mod is not None:
                print("Fitting Prophet...")
                from prophet import Prophet as ProphetClass
                ptrain = train.reset_index().rename(columns={train.index.name or "index":"ds", train.name:"y"})
                mp = ProphetClass()
                mp.fit(ptrain)
                future = mp.make_future_dataframe(periods=len(test), freq='D')
                fc = mp.predict(future)
                preds = fc['yhat'].iloc[-len(test):].values
                p_mae, p_rmse = mae(test.values, preds), rmse(test.values, preds)
                p_path = os.path.join(OUT_DIR, f"prophet_{gname}.pkl")
                if _has_joblib:
                    joblib.dump(mp, p_path)
                model_scores["Prophet"] = (p_mae, p_rmse, p_path)
                print("Prophet done:", p_mae, p_rmse)
            else:
                print("Prophet skipped.")
        except Exception as e:
            print("Prophet error:", e)

        # --- XGBoost with lag features ---
        try:
            if xgb_mod is not None:
                print("Training XGBoost...")
                df_feat = s_df.copy()
                for lag in range(1, 8):
                    df_feat[f"lag_{lag}"] = df_feat[target].shift(lag)
                df_feat = df_feat.dropna()
                X = df_feat[[c for c in df_feat.columns if c.startswith("lag_")]].values
                y = df_feat[target].values
                split_idx = int(len(X) * 0.8)
                X_train, X_test = X[:split_idx], X[split_idx:]
                y_train, y_test = y[:split_idx], y[split_idx:]
                model_xgb = xgb_mod.XGBRegressor(objective='reg:squarederror', n_estimators=100)
                model_xgb.fit(X_train, y_train)
                preds = model_xgb.predict(X_test)
                x_mae, x_rmse = mae(y_test, preds), rmse(y_test, preds)
                x_path = os.path.join(OUT_DIR, f"xgboost_{gname}.pkl")
                if _has_joblib:
                    joblib.dump(model_xgb, x_path)
                model_scores["XGBoost"] = (x_mae, x_rmse, x_path)
                print("XGBoost done:", x_mae, x_rmse)
            else:
                print("XGBoost skipped.")
        except Exception as e:
            print("XGBoost error:", e)

        # --- LSTM ---
        try:
            import tensorflow as tf_mod
            print("TensorFlow version:", tf_mod.__version__)
        except ImportError:
            tf_mod = None
            print("TensorFlow not installed.")

        try:
            if tf_mod is not None:
                print("Training LSTM (simple)...")
                from sklearn.preprocessing import MinMaxScaler
                from tensorflow.keras.models import Sequential
                from tensorflow.keras.layers import LSTM, Dense
                scaler = MinMaxScaler()
                vals = s_df[target].astype('float32').values.reshape(-1,1)
                scaled = scaler.fit_transform(vals)
                def create_seq(data, n_steps=7):
                    X, y = [], []
                    for i in range(n_steps, len(data)):
                        X.append(data[i-n_steps:i, 0])
                        y.append(data[i, 0])
                    return np.array(X), np.array(y)
                n_steps = 7
                X_all, y_all = create_seq(scaled, n_steps)
                split_idx = int(len(X_all) * 0.8)
                X_train, X_test = X_all[:split_idx], X_all[split_idx:]
                y_train, y_test = y_all[:split_idx], y_all[split_idx:]
                X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
                X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
                m = Sequential()
                m.add(LSTM(32, input_shape=(n_steps, 1)))
                m.add(Dense(1))
                m.compile(optimizer='adam', loss='mse')
                m.fit(X_train, y_train, epochs=20, batch_size=16, verbose=0)
                preds_scaled = m.predict(X_test).flatten()
                preds = scaler.inverse_transform(preds_scaled.reshape(-1, 1)).flatten()
                y_test_orig = scaler.inverse_transform(y_test.reshape(-1,1)).flatten()
                l_mae, l_rmse = mae(y_test_orig, preds), rmse(y_test_orig, preds)
                l_path = os.path.join(OUT_DIR, f"lstm_{gname}.h5")
                m.save(l_path)
                # save scaler
                if _has_joblib:
                    joblib.dump(scaler, os.path.join(OUT_DIR, f"lstm_scaler_{gname}.pkl"))
                model_scores["LSTM"] = (l_mae, l_rmse, l_path)
                print("LSTM done:", l_mae, l_rmse)
            else:
                print("LSTM skipped.")
        except Exception as e:
            print("LSTM error:", e)

        # choose best by MAE then RMSE
        if model_scores:
            best_name = sorted(model_scores.items(), key=lambda x: (x[1][0], x[1][1]))[0][0]
            best_mae, best_rmse, best_path = model_scores[best_name]
            print(f"Best model for group {gname} => {best_name} (MAE={best_mae:.4f}, RMSE={best_rmse:.4f}) path={best_path}")
            results.append({"group": gname, "best_model": best_name, "mae": best_mae, "rmse": best_rmse, "path": best_path})
        else:
            print(f"No models trained for group {gname}.")
    except Exception as e:
        print("Error processing group", grp, traceback.format_exc())

# Save summary
# -------- Save best models summary (CSV) --------
if results:
    res_df = pd.DataFrame(results)
    res_df.to_csv(BEST_MODELS_PATH, index=False)  # ✅ uses correct variable
    print(f"\n✅ Saved best model summary → {BEST_MODELS_PATH}")
else:
    print("⚠️ No models trained.")

print("\nAll artifacts →", os.listdir(OUT_DIR))
print("✅ Pipeline completed successfully.")


Loaded CSV: air_quality_data.csv  shape: (29531, 16)
Columns: ['City', 'Date', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket']
Detected datetime column: Date
Detected group column: City
Selected target column: PM2.5

--- EDA summary for target ---
              PM2.5
count  24933.000000
mean      67.450578
std       64.661449
min        0.040000
1%         7.853200
5%        13.206000
25%       28.820000
50%       48.570000
75%       80.590000
95%      193.960000
99%      311.064000
max      949.990000

Null counts per column:
City              0
PM2.5          4598
PM10          11140
NO             3582
NO2            3585
NOx            4185
NH3           10328
CO             2059
SO2            3854
O3             4022
Benzene        5623
Toluene        8041
Xylene        18109
AQI            4681
AQI_Bucket     4681
dtype: int64
Resample skipped: agg function failed [how->mean,dtype->object]
Saved cleaned dataset 

17:43:21 - cmdstanpy - INFO - Chain [1] start processing
17:43:38 - cmdstanpy - INFO - Chain [1] done processing


Prophet done: 25.080491605281534 31.2894550921331
Training XGBoost...
XGBoost done: 15.130029682435298 21.99320316188497
TensorFlow version: 2.20.0
Training LSTM (simple)...
13/13 ━━━━━━━━━━━━━━━━━━━━ 18s 657ms/step


LSTM done: 30.10061264038086 32.601863861083984
Best model for group Ahmedabad => XGBoost (MAE=15.1300, RMSE=21.9932) path=./artifacts\xgboost_Ahmedabad.pkl

Group Chennai: train=1607 test=402
Fitting SARIMAX(1,1,1) fallback...


DEBUG:cmdstanpy:cmd: where.exe tbb.dll
cwd: None


...

Prophet done: 34.84522031937735 49.64072388767631
Training XGBoost...
XGBoost done: 26.264461983088545 44.24633510177533
TensorFlow version: 2.20.0
Training LSTM (simple)...
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 152ms/step


LSTM done: 24.242023468017578 40.029808044433594
Best model for group Delhi => LSTM (MAE=24.2420, RMSE=40.0298) path=./artifacts\lstm_Delhi.h5

Group Lucknow: train=1607 test=402
Fitting SARIMAX(1,1,1) fallback...


DEBUG:cmdstanpy:cmd: where.exe tbb.dll
cwd: None


SARIMAX done: 42.289696455234484 56.21156010850941
Fitting Prophet...


DEBUG:cmdstanpy:TBB already found in load path
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: C:\Users\vsr\AppData\Local\Temp\tmp0tnv5wz2\de_v5i5p.json
DEBUG:cmdstanpy:input tempfile: C:\Users\vsr\AppData\Local\Temp\tmp0tnv5wz2\sxqtrx49.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['C:\\Users\\vsr\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\prophet\\stan_model\\prophet_model.bin', 'random', 'seed=31906', 'data', 'file=C:\\Users\\vsr\\AppData\\Local\\Temp\\tmp0tnv5wz2\\de_v5i5p.json', 'init=C:\\Users\\vsr\\AppData\\Local\\Temp\\tmp0tnv5wz2\\sxqtrx49.json', 'output', 'file=C:\\Users\\vsr\\AppData\\Local\\Temp\\tmp0tnv5wz2\\prophet_modelpema8zfj\\prophet_model-20251119175239.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:52:39 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start pro